In [1]:
import matplotlib.pyplot as plt
from atomtoolkit import ureg, Q_, util, IO
import atomtoolkit.render.lineshapes as lineshapes
from atomtoolkit.atom import Atom, Transition
from pprint import pprint
import numpy as np
import pandas as pd

In [2]:
a = Atom.load('species/Yb_II_173.atom')
a.B = Q_(5.0, 'G')

In [3]:
# pprint(a.levels.list_names())

In [4]:
# pprint(a.transitions.list_names())

In [5]:
#pprint(a.levels[0:10])

In [6]:
# t = Transition(a.levels['4f14.6s 2S1/2'], a.levels['4f14.5d 2D5/2'], A=Q_(22, 'Hz'))
# t.add_to_atom(a)
# t.set_frequency(Q_(729.476090, 'THz'))

# a.transitions[('4f14.5d 2D3/2', '4f13.(2F*<7/2>).5d.6s.(3D) 3[3/2]*1/2')].set_frequency(Q_(320.570339, 'THz'))
# a.transitions[('4f14.6s 2S1/2', '4f14.6p 2P*1/2')].set_frequency(Q_(812.128235, 'THz'))

In [7]:
#a.compute_branching_ratios('4f14.6p 2P*1/2')

In [8]:
#a.linked_levels('4f14.6p 2P*3/2')

In [9]:
#a.levels['4f13.(2F*).6s2 2F*7/2'].sublevels()

In [3]:
a.levels['4f14.6s 2S1/2 F=2']

<HFLevel(name = 4f14.6s 2S1/2 F=2, level=6120170140.0 hertz, 0 sublevels)>

In [4]:
a.transitions[('4f14.6s 2S1/2', '4f13.(2F*<7/2>).5d2.(3F) 3[3/2]*1/2')].A

80500000.0 <Unit('hertz')>

In [5]:
a.transitions[('4f14.5d 2D5/2', '4f13.(2F*<7/2>).5d.6s.(3D) 3[3/2]*3/2')].wl

2259.7456466625717 <Unit('nanometer')>

In [6]:
a.levels['4f13.(2F*<7/2>).5d.6s.(3D) 3[3/2]*3/2'].lande

1.44

In [7]:
ts = a.linked_levels('4f14.6p 2P*3/2').values()
print([t.A for t in ts if t.E_upper is a.levels['4f14.6p 2P*3/2']])
total_A = sum((t.A for t in ts if t.E_upper is a.levels['4f14.6p 2P*3/2']))
print(1 / (total_A / (2 * np.pi)))

[<Quantity(1.365e+08, 'hertz')>, <Quantity(290000.0, 'hertz')>, <Quantity(1410000.0, 'hertz')>]
4.5464437823296576e-08 / hertz


In [2]:
pd.set_option('display.width', 1000)
df = IO.load_NIST_data('yb II')
print(df.head(60))

   Level (cm-1)          Level (Hz)     J  Lande                   Configuration      Term  Percentage                 Configuration_2    Term_2  Percentage_2
0           0.0                 0.0   1/2  1.998                         4f14.6s        2S       100.0                            None                     NaN
1      21418.75   642117970978750.0   7/2  1.145                  4f13.(2F*).6s2       2F*        98.0        4f13.(2F*<7/2>).5d2.(1S)   1[7/2]*           1.0
2      31568.08   946387229754064.0   5/2  0.862                  4f13.(2F*).6s2       2F*        90.0       4f13.(2F*<7/2>)5d.6s.(3D)   3[7/2]*           7.0
3       22960.8   688347466964640.0   3/2  0.802                         4f14.5d        2D       100.0                            None                     NaN
4      24332.69   729475694485202.0   5/2  1.202                         4f14.5d        2D       100.0                            None                     NaN
5      26759.02   802215237947116.1   5/2  1.5

In [3]:
import time
for s in ['Li I', 'Na I', 'K I', 'Rb I', 'Cs I', 'Fr I', 'Be II', 'Mg II', 'Ca II', 'Sr II', 'Ba II', 'Yb II', 'Ra II','Hg II', 'Zn II', 'Al II']:
    time.sleep(1)
    try: 
        t = IO.load_NIST_data(s)
        print(f'{s} succeeded!')
        print(t.sample(5))
    except Exception as e: 
        print(f'error in {s}: {e}')

Li I succeeded!
    Level (cm-1)          Level (Hz)    J  Lande Configuration Term  Percentage Configuration_2 Term_2  Percentage_2
64      43105.42  1292267981492236.0  3/2    NaN       1s2.17p  2P*       100.0            None                  NaN
78      43296.03  1297982325534174.0  1/2    NaN       1s2.24p  2P*       100.0            None                  NaN
5       30925.38   927119568478404.1  1/2    NaN        1s2.3p  2P*       100.0            None                  NaN
26      40437.32  1212280355773256.0  5/2    NaN        1s2.6d   2D       100.0            None                  NaN
109      43412.4  1301471010367920.0  1/2    NaN       1s2.39p  2P*       100.0            None                  NaN
Na I succeeded!
    Level (cm-1)          Level (Hz)    J  Lande Configuration Term  Percentage Configuration_2 Term_2  Percentage_2
51     39734.678  1191215678545852.5  7/2    NaN        2p6.8g   2G       100.0            None                  NaN
78     40383.097  12106547911282